In [21]:
urls = ['http://httpbin.org/headers', 'http://httpbin.org/get', 'http://httpbin.org/user-agent']


In [22]:
import requests

def hello(urls):
    for url in urls:
        print(requests.get(url))

%time hello(urls)

<Response [200]>
<Response [200]>
<Response [200]>
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 1.08 s


In [31]:
#!/usr/local/bin/python3.5

import asyncio
from aiohttp import ClientSession

async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            response = await response.read()
            print(response)

loop = asyncio.get_event_loop()

tasks = []
for url in urls:
    task = asyncio.ensure_future(hello(url))
    tasks.append(task)

# 1 task
#%time loop.run_until_complete(hello())
# many tasks
%time loop.run_until_complete(asyncio.wait(tasks))                                

b'{\n  "user-agent": "Python/3.5 aiohttp/1.0.2"\n}\n'
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.5 aiohttp/1.0.2"\n  }\n}\n'
b'{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.5 aiohttp/1.0.2"\n  }, \n  "origin": "89.175.96.54", \n  "url": "http://httpbin.org/get"\n}\n'
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 390 ms


({<Task finished coro=<hello() done, defined at <ipython-input-31-55c68603a29a>:6> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-31-55c68603a29a>:6> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-31-55c68603a29a>:6> result=None>},
 set())